# King County Temperature Exploration

In [1]:
import geojson

def load_data(filename):
    """
    Loads GeoJson Data from "filename"
    """
    with open(filename) as f:
        data = geojson.load(f)
    return data

In [2]:
washington_counties_data = load_data('Washington_Counties_with_Natural_Shoreline___washsh_area.geojson')

In [3]:
def get_king_county():
    """
    Returns the GeoJson feature for King County
    """
    for feature in washington_counties_data['features']:
        if feature['properties']['COUNTY'].casefold() == 'king'.casefold():
            return feature

In [4]:

from shapely.geometry import shape, Point

king_county = get_king_county()

king_county_shape = shape(king_county['geometry'])
buffered_king_county_shape = king_county_shape.buffer(0.05)

buffered_king_county_shape_geojson = geojson.Feature(geometry=buffered_king_county_shape, properties={})

In [5]:
# https://api.weather.gov/stations?state=WA&limit=500

import requests

def get_features(geojson_data):
    return geojson_data['features']


def get_king_county_weather_stations():
    king_county_stations = []
    
    limit = 500
    state = 'WA'
    
    url = f'https://api.weather.gov/stations?state={state}&limit={limit}'
    
    response = requests.get(url)
    geojson_data = geojson.loads(response.text)
    next_url = geojson_data['pagination']['next']
    features = get_features(geojson_data)
    
    while features:
        for feature in features:
            point = shape(feature['geometry'])
            if king_county_shape.contains(point):
                king_county_stations.append(feature)
        
        response = requests.get(next_url)
        geojson_data = geojson.loads(response.text)
        next_url = geojson_data['pagination']['next']
        features = get_features(geojson_data)
        
    print(f'Found {len(king_county_stations)} stations in King County')
    return king_county_stations
    
king_county_stations = get_king_county_weather_stations()
    

Found 254 stations in King County


In [6]:
def get_temperature_at_station(station_id):
    url = f'https://api.weather.gov/stations/{station_id}/observations/latest'
    response = requests.get(url)
    if response.status_code != 200:
        return None
    geojson_data = geojson.loads(response.text)
    temperature = geojson_data['properties']['temperature']['value']
    return temperature

def get_temperatures_at_stations(stations):
    temperatures = {}
    for station in stations:
        station_id = station['properties']['stationIdentifier']
        temperature = get_temperature_at_station(station_id)
        if temperature:
            temperatures[station_id] = temperature
    return temperatures

temperatures = get_temperatures_at_stations(king_county_stations)


In [7]:
import folium

coords = tuple(reversed(king_county_shape.centroid.coords[0]))

m = folium.Map(location=coords, zoom_start=9)
folium.GeoJson(buffered_king_county_shape).add_to(m)

print(temperatures)

for station in king_county_stations:
    station_name = station['properties']['name']
    if station['properties']['stationIdentifier'] in temperatures.keys():
        temperature = temperatures[station['properties']['stationIdentifier']]
    else:
        continue
    
    coords = tuple(reversed(shape(station['geometry']).coords[0]))
    folium.Marker(location=coords, popup=f"{station_name} - {temperature}").add_to(m)

folium.LayerControl().add_to(m)

m

{'ALP31': 3.18, 'ALP44': 0.2, 'ALP55': -1.25, 'AS437': 8.33, 'AS474': 7.22, 'AU711': 7.78, 'AU957': 7.78, 'AV031': 5, 'AV093': 8.89, 'AV338': 8.89, 'AV532': 9.44, 'AV660': 8.33, 'AV836': 8.33, 'AW052': 7.78, 'AW337': 8.33, 'C1661': 8.89, 'C1943': 8.89, 'C2224': 8.33, 'C2659': 6.67, 'C3186': 9.44, 'C3683': 8.33, 'C5022': 8.33, 'C5049': 7.22, 'C5316': 6.11, 'C6259': 8.33, 'C6688': 7.78, 'C6881': 8.33, 'C7091': 10, 'C9583': 7.22, 'CBCW1': 4.4, 'CDVW1': 4.22, 'CWXW1': 2.69, 'D0372': 7.78, 'D1541': 7.78, 'D1735': 7.22, 'D3422': 8.89, 'D3884': 8.33, 'D4827': 9.44, 'D6665': 7.78, 'D6891': 7.78, 'D7537': 8.33, 'D7921': 6.67, 'D8513': 9.44, 'D9489': 7.22, 'E0259': 7.78, 'E1647': 8.89, 'E1958': 7.78, 'E2957': 8.89, 'E4799': 7.78, 'E6089': 7.78, 'E6690': 7.78, 'E6795': 9.44, 'E6986': 7.22, 'E7771': 8.33, 'E7826': 8.33, 'E7984': 8.33, 'E8046': 9.44, 'E8450': 7.78, 'E8811': 8.33, 'E9275': 7.78, 'E9714': 7.78, 'E9877': 8.89, 'ENCW1': 7.78, 'F0821': 8.89, 'F0868': 8.89, 'F2021': 7.78, 'F2316': 7.78, 